In [32]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import string
from nltk.tokenize import word_tokenize
import re

stop = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

#lowercase, remove stop words, strip punctuation
def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().strip()
    text = ' '.join([i if i not in stop and i.isalpha() else '' for i in text.lower().split()])
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text)])
    text = re.sub(r"\s{2,}", " ", text)
    return text

from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Colab Notebooks"
!cp "/content/drive/My Drive/Colab Notebooks/subreddits.txt" "subeddits.txt"

post_titles_file = open('drive/My Drive/Colab Notebooks/post_titles.txt', 'r', encoding='utf-8')
post_titles = post_titles_file.readlines()

number_samples_to_use = len(post_titles)

counter = 1
vocab = {'unknown': 0}
vocab_to_occurrences = {}
#for i in range(len(post_titles)):
for i in range(number_samples_to_use):
    post_titles[i] = clean_text(post_titles[i])
    words_in_title = post_titles[i].split()
    for word in words_in_title:
        if word not in vocab_to_occurrences.keys():
            vocab_to_occurrences[word] = 1
        else:
            vocab_to_occurrences[word] = vocab_to_occurrences[word] + 1
#for i in range(len(post_titles)):
for i in range(number_samples_to_use):
    post_titles[i] = clean_text(post_titles[i])
    words_in_title = post_titles[i].split()
    for word in words_in_title:
        if word not in vocab_to_occurrences.keys():
            vocab_to_occurrences[word] = 1
        else:
            vocab_to_occurrences[word] = vocab_to_occurrences[word] + 1
        if word not in vocab.keys():
            if vocab_to_occurrences[word] >= 30:
                vocab[word] = counter
                counter += 1

#might want to preprocess so that words which hardly appear do not come up, currently 28797 words
print(len(vocab))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
' badRNN.ipynb'        post_titles.txt	 Untitled
 hw1.ipynb	       subreddits.txt	 Untitled0.ipynb
 mnist_pytorch.ipynb   TextOnly.ipynb	 Untitled1.ipynb
2420


In [48]:
import torch
import random

subeddits_file = open('drive/My Drive/Colab Notebooks/subreddits.txt', 'r', encoding='utf-8')
subreddits = subeddits_file.readlines()
subreddit_to_number = {}
number_to_subreddit = {}

vocab_size = len(vocab)

subr_counter = 0
#for i in range(len(subreddits)):
for i in range(number_samples_to_use):
    if subreddits[i] not in subreddit_to_number.keys():
        subreddit_to_number[subreddits[i]] = subr_counter
        number_to_subreddit[str(subr_counter)] = subreddits[i]
        subr_counter += 1
num_subreddits = len(subreddit_to_number)
print(num_subreddits)


formatted_data = []
#for i in range(len(subreddits)):
for i in range(number_samples_to_use):
    cur_title = post_titles[i]
    cur_subreddit = subreddits[i]
    datapoint = torch.zeros((vocab_size, 1))
    for word in cur_title:
        if word in vocab.keys():
            datapoint[vocab[word]] = 1
    datapoint = datapoint.transpose(0, 1)
    label = subreddit_to_number[cur_subreddit]
    label = torch.LongTensor([label])
    tuple_data = (datapoint, label)
    formatted_data.append(tuple_data)

#randomly rearrange
#for i in range(2*len(formatted_data)):
#    first_index = random.randint(0, len(formatted_data ) - 1)
#    second_index = random.randint(0, len(formatted_data) - 1)

#    temp = formatted_data[first_index]
#    formatted_data[first_index] = formatted_data[second_index]
#    formatted_data[second_index] = temp

#    temp = subreddits[first_index]
#    subreddits[first_index] = subreddits[second_index]
#    subreddits[second_index] = temp

num_subset = 20000
formatted_data = formatted_data[:num_subset]

20


In [34]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class FastWordModel(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.lin1 = nn.Linear(vocab_size, 512)
        self.lin2 = nn.Linear(512, num_classes)
        #1 by vocab size -> 1 by 512 -> 1 by num_classes

    def forward(self, x):
        x = F.leaky_relu(self.lin1(x))
        return F.leaky_relu(self.lin2(x))
    
    def loss(self, prediction, label):
        loss_val = F.cross_entropy(prediction, label)
        return loss_val

In [49]:
model = FastWordModel(vocab_size, num_subreddits)
LEARNING_RATE = 0.01
MOMENTUM = 0
WEIGHT_DECAY = 0
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)    

#try predictions
def predict(model, data):
    counter = 0
    num_correct = 0
    with torch.no_grad():
        for x,y in data:
            output = model(x)
            subreddit_number = torch.argmax(output).numpy()
            #post_title_file = open('data/post_titles.txt', 'r', encoding='utf-8')
            #post_titles = post_title_file.readlines()
            #subreddit_file = open('drive/My Drive/Colab Notebooks/subreddits.txt', 'r', encoding='utf-8') 
            #subreddits = subreddit_file.readlines()
            #if subreddits[counter] == number_to_subreddit[str(subreddit_number)]:
            #    num_correct += 1


            if subreddit_number == y[0].numpy():
                num_correct += 1
            counter += 1
        print(num_correct)
        print(counter)
        print(str(1.0*num_correct/counter))
#predict(model, test_data)
  
def train(model, data, test, optimizer):
    num_epochs = 20
    counter = 0
    for i in range(num_epochs):
        running_loss = 0
        for x,y in data:
            optimizer.zero_grad()
            #not sure why this tensor nesting happened
            output = model(x)[0]
            #import pdb; pdb.set_trace()
            loss = model.loss(torch.unsqueeze(output, 0), y)
            #import pdb; pdb.set_trace()
            loss.backward()
            optimizer.step()
            running_loss += loss
            counter += 1
        print("epoch number " + str(i) + " has running loss " + str(running_loss))
        predict(model, test)
        counter = 0

train_data = formatted_data[0:int(0.8*len(formatted_data))]
test_data = formatted_data[int(0.8*len(formatted_data)):]

train(model, train_data, test_data, optimizer)
print("trained")

epoch number 0 has running loss tensor(5443.4355, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 1 has running loss tensor(4355.6094, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 2 has running loss tensor(3445.0520, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 3 has running loss tensor(2870.8315, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 4 has running loss tensor(2702.6672, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 5 has running loss tensor(2545.5081, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 6 has running loss tensor(2499.0618, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 7 has running loss tensor(2474.6760, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 8 has running loss tensor(2487.0945, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 9 has running loss tensor(2421.7690, grad_fn=<AddBackward0>)
1037
4000
0.25925
epoch number 10 has running loss tensor(2331.8665, grad_fn=<AddBackward0>)
1037


In [ ]:

0.28025

0.29825

0.30225

0.30525

0.3085

0.308

0.3085

0.30825

0.31325

0.3155

0.3135

0.31425

0.31425

0.31525

0.31575

0.31675

0.31475

0.317

0.31575

0.31475
